In [2]:
import requests
import json
import datetime
import os
from dotenv import load_dotenv
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

# .env ファイルから環境変数を読み込む
load_dotenv()

def get_lat_lon(prefecture, city):
    """
    県名と市名を入力すると、緯度と経度を出力する関数

    Args:
        prefecture (str): 県名（例: "東京都"）
        city (str): 市名（例: "新宿区"）

    Returns:
        tuple: (緯度, 経度) のタプル。見つからない場合は None を返す。
    """
    address = f"{prefecture}{city}"
    geolocator = Nominatim(user_agent="weather_app")
    try:
        location = geolocator.geocode(address, timeout=5)  # タイムアウトを設定
        if location:
            return (location.latitude, location.longitude)
        else:
            print(f"'{address}' の緯度経度が見つかりませんでした。")
            return None
    except GeocoderTimedOut:
        print("タイムアウトエラーが発生しました。")
        return None
    except GeocoderUnavailable:
        print("ジオコーダが利用できません。")
        return None
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None

def get_weather_forecast_by_coords(lat, lon, api_key):
    """
    指定した緯度・経度の場所の現在時刻以降の3時間ごとの天気予報を取得する
    
    Parameters:
    -----------
    lat : float
        緯度
    lon : float
        経度
    api_key : str
        OpenWeather APIのAPIキー
    
    Returns:
    --------
    dict
        フォーマット済みの天気予報データ
    """
    
    utc_now = datetime.datetime.utcnow()
    start_time = utc_now + datetime.timedelta(hours=9)  # UTCから日本時間（+9時間）
    today = start_time.replace(hour=23, minute=59, second=59)  # 今日の終わり
    # APIのエンドポイントURL
    url = "https://api.openweathermap.org/data/2.5/forecast"
    
    # パラメータの設定
    params = {
        "lat": lat,  # 緯度
        "lon": lon,  # 経度
        "appid": api_key,  # APIキー
        "units": "metric",  # 摂氏で温度を取得
        "lang": "ja"  # 日本語で結果を取得
    }
    
    # APIリクエストを送信
    response = requests.get(url, params=params)
    
    # レスポンスが成功した場合
    if response.status_code == 200:
        data = response.json()
            
        # 必要なデータだけを抽出
        result = {
            "city": data["city"]["name"],
            "country": data["city"]["country"],
            "coordinates": {
                "latitude": lat,
                "longitude": lon
            },
            "forecasts": []
        }
        
        # 3時間ごとの予報データを処理
        for forecast in data["list"]:
            # UTC時間からJST(+9時間)に変換
            utc_time = datetime.datetime.utcfromtimestamp(forecast["dt"])
            forecast_time = utc_time + datetime.timedelta(hours=9)  # UTCから日本時間へ
            
            # 指定した開始時刻以降のデータのみ処理
            if start_time <= forecast_time <= today:
                # 降水量の取得（存在しない場合は0）
                rain_3h = forecast.get("rain", {}).get("3h", 0)
                
                # 必要なデータを抽出
                forecast_data = {
                    "datetime": forecast_time.strftime("%Y-%m-%d %H:%M:%S"),
                    "weather": {
                        "main": forecast["weather"][0]["main"],
                        "description": forecast["weather"][0]["description"],
                        "icon": forecast["weather"][0]["icon"]
                    },
                    "temperature": forecast["main"]["temp"],
                    "feels_like": forecast["main"]["feels_like"],
                    "precipitation": rain_3h
                }
                
                result["forecasts"].append(forecast_data)
        
        return result
    else:
        print(f"エラー: {response.status_code}")
        return None

def save_to_json(data, filename):
    """
    データをJSONファイルに保存する
    
    Parameters:
    -----------
    data : dict
        保存するデータ
    filename : str
        保存先のファイル名
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"データを {filename} に保存しました。")


# APIキーを環境変数から取得
api_key = os.getenv("OPENWEATHER_API_KEY")
if not api_key:
    api_key = input("OpenWeather APIキーを入力してください: ")

# 県名と市名の入力
prefecture = input("県名を入力してください（例：東京都）: ")
city = input("市名を入力してください（例：新宿区）: ")

# 緯度経度の取得
coordinates = get_lat_lon(prefecture, city)

if not coordinates:
    print("緯度経度の取得に失敗したため、処理を終了します。")
else:
    latitude, longitude = coordinates
    print(f"{prefecture}{city} の緯度: {latitude}, 経度: {longitude}")
    
    # 開始時刻の入力（オプション）
    use_custom_time = input("特定の開始時刻を指定しますか？(y/n): ").lower() == 'y'
    start_time = None
    
    if use_custom_time:
        date_str = input("開始日を入力してください (YYYY-MM-DD): ")
        time_str = input("開始時刻を入力してください (HH:MM): ")
        try:
            start_time = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
        except ValueError:
            print("日時の形式が正しくありません。現在時刻を使用します。")
            start_time = None
    
    # 天気データの取得
    weather_data = get_weather_forecast_by_coords(latitude, longitude, api_key)
    
    if weather_data:
        # ファイル名の設定（場所と日時を含める）
        location_name = f"{prefecture}_{city}".replace(' ', '_')
        filename = f"weather_{location_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.json"
        
        # JSONファイルに保存
        save_to_json(weather_data, filename)

大阪府堺市 の緯度: 34.5737364, 経度: 135.4828866
データを weather_大阪府_堺市_20250503_1703.json に保存しました。
